## Закупки

In [1]:
import pandas as pd
import numpy as np

### Загрузка данных

In [2]:
tenders = pd.read_csv("tenders/tenders.csv", dtype= {"lot_name":np.str,
                                                     "key": np.str,
                                                     "inn_num": np.str,
                                                     "okpd2": np.str,
                                                     "purchase_method": np.str,
                                                     "open_dt": np.str,
                                                     "close_dt": np.str,
                                                     "organizer_region": np.str,
                                                     "initial_price": np.float64})

In [3]:
tenders.head()

,lot_name,key,inn_num,okpd2,purchase_method,open_dt,close_dt,organizer_region,initial_price
0,Аренда транспортного средства с экипажем. Терр...,32008811985_1,6952315887,77.12.1,Закупка у единственного поставщика,2019-12-30,2019-12-30,Тверская область,4499979.000
1,Аренда транспортных средств с экипажем (КАМАЗ ...,32008810970_1,6952315887,77.12,Закупка у единственного поставщика,2019-12-31,2019-12-31,Тверская область,2171892.000
2,Разработка раздела проектной документации «Про...,32008809236_1,1435142972,71.12.11,Закупка у единственного поставщика,2019-12-30,2019-12-30,Республика Саха (Якутия),197297.000
3,"Аренда специальной техники с экипажем, террито...",32008807943_1,6952315887,77.12.1,Закупка у единственного поставщика,2019-12-30,2019-12-30,Тверская область,600000.000
4,"Аренда специальной техники с экипажем, террито...",32008812309_1,6952315887,77.12.1,Закупка у единственного поставщика,2019-12-30,2019-12-30,Тверская область,1377714.375


In [3]:
participants = pd.read_csv("tenders/participants.csv" , dtype= {"inn_num": np.str, "key": np.str, "winner_flg": np.int} )

In [4]:
participants.head()

,inn_num,key,winner_flg
0,0000000000,31908636260_1,1
1,0000000000,31908564969_1,1
2,7805287581,0368400000219000451_1,0
3,5043051626,0368400000219000451_1,0
4,7717784658,0368400000219000451_1,1


In [5]:
participants_okved = pd.read_csv("tenders/participants_okved.csv", dtype= {"inn_num": np.str, "okved_main1": np.str, "okved_main0": np.str} )

In [6]:
participants_okved

,inn_num,okved_main1,okved_main0
0,0000000000,NaN,NaN
1,7805287581,['46.69.5'],"['18.1', '33.14', '33.20', '41.2', '43.2', '43..."
2,5043051626,['46.90'],"['25.62', '25.99.29', '27.12', '27.90', '28.13..."
3,7717784658,['73.11'],"['33.12', '33.14', '33.20', '46.18', '46.19', ..."
4,7806239260,['46.51'],"['46.41.1', '46.66', '46.90', '62.01', '62.02'..."
...,...,...,...
293495,7810625250,['71.12.12'],"['26.51.5', '26.51.8', '30.11', '33.12', '33.1..."
293496,6444009038,"['26.51.7', '26.51.8']","['01.19.2', '01.30', '10.71', '25.72', '26.20'..."
293497,2635001120,['26.51.7'],"['20.60.1', '20.60.2', '22.19.7', '22.21', '22..."
293498,615509605586,['49.41.1'],"['49.41.2', '49.41.3', '77.39.11']"


### Подготовка данных

Оставляем только определенные виды закупок

In [7]:
tenders["purchase_method"].value_counts()

Электронный аукцион                   1540817
Запрос котировок                       227426
Закупка у единственного поставщика      90041
Запрос предложений                      42621
Открытый конкурс                        39422
Открытый запрос котировок               15775
Прочее                                  12042
Открытый запрос предложений              9885
Конкурс                                  9392
Конкурс с ограниченным участием          9297
Предварительный отбор                    5824
Аукцион                                  3718
Конкурентные переговоры                   873
Открытый аукцион                          464
Маркетинговое исследование                346
Закрытый запрос котировок                 143
Мониторинг цен                            124
Закрытый аукцион                           43
Закрытый запрос предложений                40
Двухэтапный конкурс                        18
Единственный источник                       4
Name: purchase_method, dtype: int6

In [17]:
filtered_tenders = tenders[tenders["purchase_method"].isin(set(["Электронный аукцион",
                                        "Запрос котировок",
                                        "Запрос предложений",
                                        "Открытый конкурс",
                                        "Открытый запрос котировок",
                                        "Открытый запрос предложений",
                                        "Конкурс"]))]

In [18]:
# будем использовать окпд не ниже второго уровня
filtered_tenders.loc[:,"okpd2"]=filtered_tenders["okpd2"].apply(lambda df: ".".join(df.split(",")[0].split(".",2)[:2])).values

In [28]:
# в качестве объектов используем пару окпд + регион
filtered_tenders.loc[:,"items"] = (
np.core.defchararray.add(filtered_tenders["organizer_region"].values.astype(np.str) ,filtered_tenders["okpd2"].values.astype(np.str)))

In [89]:
filtered_tenders = filtered_tenders[~filtered_tenders["organizer_region"].isna()]

Вытащим основной оквэд для каждого участника, удалим участнииков без оквэда

In [40]:
participants_okved = participants_okved[~participants_okved.okved_main1.isna()]

In [44]:
participants_okved.loc[:,"okved"] = (
participants_okved["okved_main1"].apply(lambda df: df[2:-2].split(",")[0].replace("'","")))

Соединим все таблички в одну 

In [90]:
full_table=pd.merge(pd.merge(participants,filtered_tenders[["items","key","close_dt","open_dt"]],left_on="key",right_on="key",how="inner"),
                    participants_okved[["inn_num","okved"]],left_on="inn_num",right_on="inn_num",how="inner")

In [91]:
full_table.loc[:,"region_code"]=full_table["inn_num"].apply(lambda df: df[:2])

In [92]:
full_table = full_table[(~full_table["items"].isna()) & (~full_table["inn_num"].isna()) & (full_table["inn_num"]!="0000000000")]

In [93]:
# таргетом считаем любые участия в торгах
full_table.loc[:,"winner_flg"] = 1

In [94]:
full_table.head(5)

,inn_num,key,winner_flg,items,close_dt,open_dt,okved,region_code
0,7805287581,0368400000219000451_1,1,Ульяновская область26.20,2020-01-15,2019-12-28,46.69.5,78
1,7805287581,0368400000219000428_1,1,Ульяновская область26.20,2020-01-13,2019-12-24,46.69.5,78
2,7805287581,0368400000219000041_1,1,Ульяновская область13.93,2019-07-19,2019-07-11,46.69.5,78
3,7805287581,0368400000219000042_1,1,Ульяновская область31.01,2019-07-19,2019-07-11,46.69.5,78
4,7805287581,0368400000219000043_1,1,Ульяновская область26.20,2019-07-19,2019-07-11,46.69.5,78


In [95]:
tenders_by_day = full_table[["items","open_dt","key"]].drop_duplicates().groupby(["items","open_dt"]).count()

In [96]:
np.percentile(tenders_by_day["key"],99)

13.0

In [97]:
np.percentile(tenders_by_day["key"],95)

6.0

In [98]:
np.percentile(tenders_by_day["key"],90)

4.0

In [99]:
tenders_by_day.iloc[np.argsort(tenders_by_day["key"])[-20:]]

key
items                                            open_dt        
Москва32.50                                      2019-04-30  108
                                                 2019-05-27  113
Республика Башкортостан32.50                     2019-03-28  116
Москва32.50                                      2019-11-28  117
Москва71.1                                       2019-11-25  117
Московская область80.10                          2019-11-18  122
Ханты-Мансийский Автономный округ - Югра АО68.10 2019-11-26  123
Москва32.50                                      2019-12-09  124
                                                 2019-11-26  124
                                                 2019-12-30  124
                                                 2019-12-27  127
                                                 2019-04-29  127
Санкт-Петербург32.50                             2019-12-30  127
Москва32.50                                      2019-11-22  133
                                                 2019-11-20  133
Ханты-Мансийский Автономный округ - Югра АО68.10 2019-11-22  136
                                                 2019-11-27  140
                                                 2019-11-28  140
Москва32.50                                      2019-09-30  146
Ханты-Мансийский Автономный округ - Югра АО68.10 2019-11-25  193

### Обучение моделей

In [79]:
from replay.session_handler import State

spark = State().session
spark

In [103]:
from replay.data_preparator import DataPreparator

log = DataPreparator().transform(
    data=full_table,
    columns_names={
        "user_id": "inn_num",
        "item_id": "items",
        "relevance": "winner_flg",
        "timestamp": "close_dt"
    }
)

In [104]:
from replay.filters import min_entries
log_filtered = min_entries(log, 5)

In [105]:
from replay.splitters import UserSplitter, RandomSplitter

splitter = UserSplitter(
    drop_cold_items=True,
    drop_cold_users=True,
    item_test_size=1,
    seed=1234,
    shuffle=False,
)
train, test = splitter.split(log)
(
    train.count(), 
    test.count()
)

(4318173, 197022)

In [106]:
from pyspark.sql import functions as sf
from replay.utils import get_top_k_recs, convert2spark

In [108]:
from replay.metrics import HitRate, NDCG
from replay.experiment import Experiment

metrics = Experiment(test, {NDCG(): 4, 
                            HitRate(): 4},calc_median=True,calc_conf_interval=0.95 )

### Популярный рекомендатель

In [107]:
recs_all=train.groupBy(["user_id","item_id"]).agg(sf.count("relevance").alias("relevance"))

In [ ]:
#recs_pop_okved=convert2spark(full_table[["okved", "region_code", "open_dt"]].drop_duplicates().groupby(["items","open_dt"]).count())

In [131]:
metrics.add_result("Popular", get_top_k_recs(recs_all, 4))
metrics.results.loc["Popular"]

06-Aug-20 09:32:56, replay, WARNING: Значение метрики может быть неожиданным: пользователи в recommendations и ground_truth различаются!
06-Aug-20 09:33:37, replay, WARNING: Значение метрики может быть неожиданным: пользователи в recommendations и ground_truth различаются!
06-Aug-20 09:34:21, replay, WARNING: Значение метрики может быть неожиданным: пользователи в recommendations и ground_truth различаются!
06-Aug-20 09:35:02, replay, WARNING: Значение метрики может быть неожиданным: пользователи в recommendations и ground_truth различаются!
06-Aug-20 09:35:52, replay, WARNING: Значение метрики может быть неожиданным: пользователи в recommendations и ground_truth различаются!
06-Aug-20 09:36:46, replay, WARNING: Значение метрики может быть неожиданным: пользователи в recommendations и ground_truth различаются!


HitRate@4                       0.514235
HitRate@4_median                1.000000
HitRate@4_0.95_conf_interval    0.003333
NDCG@4                          0.402731
NDCG@4_median                   0.430677
NDCG@4_0.95_conf_interval       0.002835
Name: Popular, dtype: float64

### ADMM-SLIM

In [109]:
from replay.models import ADMMSLIM

slim = ADMMSLIM(4,1000)

In [110]:
%%time

slim.fit(log=train)

04-Aug-20 11:42:19, replay, DEBUG: Начало обучения ADMMSLIM
04-Aug-20 11:42:19, replay, DEBUG: Предварительная стадия обучения (pre-fit)
04-Aug-20 11:42:23, replay, DEBUG: Основная стадия обучения (fit)
04-Aug-20 11:42:42, replay, DEBUG: Построение модели ADMM SLIM
04-Aug-20 11:42:44, replay, DEBUG: Матриица Грама
04-Aug-20 11:42:48, replay, DEBUG: Поиск обратной матрицы
04-Aug-20 11:45:37, replay, DEBUG: Основной  расчет
04-Aug-20 11:48:27, replay, INFO: Итерация: 1. primal gap: 83.04; dual gap:  1.3041e+07; rho: 500.0
04-Aug-20 11:49:19, replay, INFO: Итерация: 2. primal gap: 14.4; dual gap:  3.1523e+06; rho: 250.0
04-Aug-20 11:50:06, replay, INFO: Итерация: 3. primal gap: -12.833; dual gap:  4.3446e+05; rho: 125.0
04-Aug-20 11:50:54, replay, INFO: Итерация: 4. primal gap: 19.514; dual gap:  2.3017e+04; rho: 62.5
04-Aug-20 11:51:42, replay, INFO: Итерация: 5. primal gap: 24.164; dual gap:  -7.427; rho: 62.5
04-Aug-20 11:52:28, replay, INFO: Итерация: 6. primal gap: 22.38; dual gap:  

CPU times: user 10h 37min 22s, sys: 1h 45min 55s, total: 12h 23min 17s
Wall time: 23min 41s


In [111]:
%%time
recs = slim.predict(
    k=4,
    users=test.select('user_id').distinct(),
    items=train.select('item_id').distinct(),
    log=train,
    filter_seen_items=False
)

04-Aug-20 12:06:00, replay, DEBUG: Начало предикта ADMMSLIM


CPU times: user 15.3 s, sys: 3.57 s, total: 18.8 s
Wall time: 40 s


In [112]:
%%time
metrics.add_result("SLIM", recs)
metrics.results.loc["SLIM"]

04-Aug-20 12:13:13, replay, WARNING: Значение метрики может быть неожиданным: пользователи в recommendations и ground_truth различаются!
04-Aug-20 12:13:58, replay, WARNING: Значение метрики может быть неожиданным: пользователи в recommendations и ground_truth различаются!
04-Aug-20 12:14:42, replay, WARNING: Значение метрики может быть неожиданным: пользователи в recommendations и ground_truth различаются!
04-Aug-20 12:15:26, replay, WARNING: Значение метрики может быть неожиданным: пользователи в recommendations и ground_truth различаются!
04-Aug-20 12:16:22, replay, WARNING: Значение метрики может быть неожиданным: пользователи в recommendations и ground_truth различаются!
04-Aug-20 12:17:18, replay, WARNING: Значение метрики может быть неожиданным: пользователи в recommendations и ground_truth различаются!


CPU times: user 300 ms, sys: 46.2 ms, total: 346 ms
Wall time: 11min 30s


HitRate@4                       0.231903
HitRate@4_median                0.000000
HitRate@4_0.95_conf_interval    0.001874
NDCG@4                          0.166233
NDCG@4_median                   0.000000
NDCG@4_0.95_conf_interval       0.001434
Name: SLIM, dtype: float64

### MultVAE

In [123]:
from replay.models import MultVAE
vae=MultVAE(epochs=100,learning_rate=0.01,latent_dim=200,encoder_dims=[600])

In [124]:
%%time

vae.fit(log=train)

05-Aug-20 11:02:45, replay, DEBUG: Начало обучения MultVAE
05-Aug-20 11:02:45, replay, DEBUG: Предварительная стадия обучения (pre-fit)
05-Aug-20 11:02:48, replay, DEBUG: Основная стадия обучения (fit)
05-Aug-20 11:03:10, replay, DEBUG: Составление батча:
/home/avavasilyev/sponge-bob-magic/venv/lib/python3.7/site-packages/pyarrow/util.py:39: FutureWarning: pyarrow.open_stream is deprecated as of 0.17.0, please use pyarrow.ipc.open_stream instead
  warnings.warn(msg, FutureWarning)
05-Aug-20 11:03:12, replay, DEBUG: Обучение модели
05-Aug-20 11:04:09, replay, DEBUG: Epoch[1] current loss: 214.40842
05-Aug-20 11:04:12, replay, DEBUG: Epoch[1] validation average loss: 154.74897
05-Aug-20 11:05:08, replay, DEBUG: Epoch[2] current loss: 143.60321
05-Aug-20 11:05:11, replay, DEBUG: Epoch[2] validation average loss: 109.81307
05-Aug-20 11:06:06, replay, DEBUG: Epoch[3] current loss: 114.37274
05-Aug-20 11:06:09, replay, DEBUG: Epoch[3] validation average loss: 98.34301
05-Aug-20 11:07:05, rep

CPU times: user 6h 23min 59s, sys: 13h 1min 27s, total: 19h 25min 27s
Wall time: 52min


In [127]:
%%time
recs = vae.predict(
    k=4,
    users=test.select('user_id').distinct(),
    items=train.select('item_id').distinct(),
    log=train,
    filter_seen_items=False
)

06-Aug-20 07:35:26, replay, DEBUG: Начало предикта MultVAE
06-Aug-20 07:35:55, replay, DEBUG: Предсказание модели


CPU times: user 16.6 s, sys: 4.87 s, total: 21.5 s
Wall time: 46min 30s


In [128]:
%%time
metrics.add_result("VAE", recs)
metrics.results.loc["VAE"]

CPU times: user 411 ms, sys: 150 ms, total: 561 ms
Wall time: 50min 41s


HitRate@4                       0.271756
HitRate@4_median                0.000000
HitRate@4_0.95_conf_interval    0.001964
NDCG@4                          0.173075
NDCG@4_median                   0.000000
NDCG@4_0.95_conf_interval       0.001348
Name: VAE, dtype: float64

In [132]:
metrics.results

,HitRate@4,HitRate@4_median,HitRate@4_0.95_conf_interval,NDCG@4,NDCG@4_median,NDCG@4_0.95_conf_interval
SLIM,0.231903,0.0,0.001874,0.166233,0.000000,0.001434
VAE,0.271756,0.0,0.001964,0.173075,0.000000,0.001348
Popular,0.514235,1.0,0.003333,0.402731,0.430677,0.002835


### Вывод

Обычный поплуярный алгоритм достаточно хорош, хотя и дает рекомендации не всем пользователям в нужном количестве (можно виидеть что довериительный интервал для него шире, а это значиит что в оценке участвует меньше пользователей). На первых этапах вполне нормально использовать именно его. Это в том числе связано с тем, что нам необходимо рекомендовать и те пары регион+ОКПД, в которых клииент уже участвовал.